## Tutorial 2) Readout sequences

## 0. Introduction

In the first tutorial sequence writing and parameterization with configuration files was demonstrated.
This enables user to apply and quantify any waveform on the instrument outputs.

Playing arbitrary waveforms is one part of qubit experimetns, reading them out is equally crucial.
In this tutorial we explain the concept of `ReadSequence`s and how to use them. 

The tutorial is structured in the following way:
1. The basic `ReadSequence` architecture
2. Configuring a dummy readout sequence
3. Compiling the Sequence to QUA code
4. Parameter sweeps and measurements
5. Scaling readout sequences to bigger systems

Chapter 1-4 demonstrates how a `ReadSequence` is built, how to configure and how to compile it. Part 4 connects this with sweeps of arbitrary parameters and part 5 demonstrates how to scale up the examined systems without altering the ReadSequence class.

## 1. The basic `ReadSequence` architecture

`ReadSequence`\
The ReadSeqeunce is a child class of the SubSequence which we exlpored in Tutorial 1. Arbitrary waveforms can still be played, parameterized and swept.
The ReadSequence is meant to handle measurement sequences and describe them in a device agnostic way. For this the classes in the rest of this list are introduced. They all relate to a single given ReadSequence.

`Signal`\
Signals represent one or more measurement results from a single physical entity (e.g qubit, SET, quantum dot etc.) A ReadSequence can have an arbitrary number of Signal instances.

`Observable`\
Each observable handles a single result that is being accquired during the execution of a ReadSequence. Observables store the qua variables that store measurement results temporarily and the streams they are saved to. Each result is assigned to a single signal instance.

`ReadoutPoint`\
A readout point describes a direct readout of a given quantum element at a differnet 'point' (e.g in voltage or frequency space).
Qua commands describing this measurement are given in the `qua_measure` method.
Per Readout point, multiple observables can be instruduced whose FPGA variables are automatically declared, assigned and saved to the correct stream by this class.

`AbstractReadout`\
The AbstractReadout works similarly to the ReadoutPoint but gives you all the freedom. Qua commands can be executed, arbitrary arguments like observables from other AbstractReadouts or even ReadoutPoints can be passed as arguments to process them further. in comparison to the ReadoutPoint, the results (observables) of an AbstractReadout can be assigned to ANY given signal.

This might seem a bit abstract on the first glace but will be a lot more clear after looking how this works in action.

<img src="images/readout_sequence_scheme.svg">

This scheme can be further visualized by comparing this with the configuration we will use in this tutorial.

## 2. Configuring a dummy readout sequence

### 2.1 The three parts of the configuration

Below you an example configuration is given. I might seem overwhelming at first but can easily be sliced into logical blocks.

The given dictionairy has **three** keys.

1) The 'parameters' section acts identically to the configuration of a `SubSequence` that we discussed before. Parameters are created according to provided names, values, units etc.
2) The 'signals' section defines the physical entities we want to measure that are grouped into `Signal`s. To define a signal, the involved readout elements of the quantum machine need to be provided as well as its `ReadoutPoints` that are being used in the measurement.
3) The last key is the 'readout_groups' within which multiple logical groups are defined that again contain configurations for `AbstracReadout`s. Those groups are convenient if you want to execute many readout operations at the same time. An abstract readout is simply configured by giving the desired **'method'** (1), a **name** under which the result should be stored on the signal (2) and the **arguments** that are required for the given 'method' (3).


In [1]:
readout_sequence_config = {
    'parameters': {
        't_between_measurements': {
            'value': 50,
            'unit': 'cycles',
        }
    },
    'signals':{
        'qubit1':{
            'elements': {
                'sensor1': 'readout_element',
            },
            'readout_points': {
                'reference': {
                    'method': 'average',
                    'desc':'reference point',
                    'observables': ['I', 'Q', 'IQ'],
                    'save_values': True
                },
                'read': {
                    'method': 'average',
                    'desc': 'redout point',
                    'observables': ['I', 'Q', 'IQ'],
                    'save_values': True
                }
            }
        },
    },
    'readout_groups': {
        'difference': {
            'qubit1__diff': {
                'method': 'difference',
                'name': 'diff',
                'args': {
                    'signal': 'qubit1',
                    'minuend': 'qubit1.reference.sensor1_IQ',
                    'subtrahend': 'qubit1.read.sensor1_IQ',
                },
            },
        }
    },
}

### 2.2 Writing custom `Readsequence`s

In the next step we will have a look at a custom `ReadSequence` that takes a 'reference' and a 'read' measurement for each signal that has those `ReadPoint`s provided.
After that the difference between those accquired results is calculated (in real time on the FPGA) and saved in the respective buffer.

In [2]:
from example_configs.dummy_sample import dummy_sample
from example_sequences.dummy_readout import DummyReadout

2024-04-12 13:25:51,606 - qm - INFO     - Starting session: ce62e7b2-5eb3-489b-a95e-f084c96c2e1a


Let us have a look at the source code. Writing a custom class can be quite simple as only two methods need to be provided:

`__init__`:\
This method is called when the given class is instanciated. We need to provide a name, the used `Sample` and a configuration as given above.
Within this method the parent class `ReadSequence` '__init__' method is being called as well with super(). All arguments are forwarded to the parent and the available types of `ReadoutPoint`s and `AbstractReadout`s are being passed as well. All keys provided in those dictionaries can be used in the 'method' section of the sequence configuration discussed above.

`qua_sequence`:\
This method contains qua commands to be executed on the hardware defining our measurement. In this example not a lot of explicit qua code can be seen but rather method calls to `qua_measure_and_save`. Those helper classes store the respective qua code in their own methods. 
It can be seen that explicit FPGA variable declarion and stream/ memory management is not required. This is all handled automatically by the given observables. Rememer, an `Observable` is always responsible for a single measurement result and stores its qua variables, and data streams and puts them in the correct place in the qua program.

In [3]:
DummyReadout??

Init signature: DummyReadout(*args: 'Any', **kwargs: 'Any') -> 'Any'
Source:        
class DummyReadout(ReadSequence):
    """
    Class containing parameters and sequence for a dummy read sequnce
    """
    def __init__(
            self,
            name: str,
            sample: Sample,
            sequence_config: dict,
        ):
        """
        Constructor method for 'DummyReadout' class
        
        Args:
            name (str): name of the sequence
            sample (Sample): Sample being used
            sequence_config (dict): Dict configuring sequence
        """
        super().__init__(
            name, sample, sequence_config,
            {'difference': Difference},
            {'average': Average}
            )

    def qua_sequence(self):
        """QUA sequence to perform mixed down up initialization"""
        qua.align()

        ref_points = [x for x in self.readout_points.values() if x.point_name == 'reference']
        for readout_point in ref_points:
 

In [4]:
dummy_readout = DummyReadout('dummy_readout', dummy_sample, readout_sequence_config)

With the discussed configuration, a sample object and the dummy readout class above, we instanciated a DummyReadout.

### 2.3 Attributes and helpers of `ReadSequence`

In this section we will explore the attributes and helper of a `ReadSeqeunce` that a user can interact with after instanciation with a configuration file. The given examples uses the `DummyReadout` from above and the example configuration from the section before that. 

For the sake of readability jupyter inline tools from rich are imported.

In [5]:
%load_ext rich

As in tutorial 1, an `ArbokDriver` and an empty `Sequence` is created to which we add our dummy_readout.

In [6]:
from arbok_driver import ArbokDriver, Sample, Sequence

In [7]:
qm_driver = ArbokDriver('qm_driver', dummy_sample)
dummy_sequence = Sequence('dummy_squence', dummy_sample)
qm_driver.add_sequence(dummy_sequence)
dummy_sequence.add_subsequence(dummy_readout)

The easiest way to quickly inspect any qcodes instrument is to call its `print_readable_snapshot` method.

#### 2.3.1 Parameters

First of all, a standard `SequenceParameter` can be found. This is the one we defined in the 'parameters' section of the configuration above as 't_between_measurements' and behaves exactly as a parameter in a simple `SubSequence`.

Then many parameters with rather lengthy names can be found whose value is given as 'Not available'. Those are the 'GettableParameters' we will be able to measure once the program is run on the hardware. Their names are always unique since sequence and signal names have to be unique as well. Therefore those names are used as well to define streams within qua.

In [8]:
dummy_readout.print_readable_snapshot()

dummy_readout:
	parameter                                   value
--------------------------------------------------------------------------------
IDN                                          :	None 
dummy_readout__qubit1__diff                  :	Not available 
dummy_readout__qubit1__read__sensor1_I       :	Not available 
dummy_readout__qubit1__read__sensor1_IQ      :	Not available 
dummy_readout__qubit1__read__sensor1_Q       :	Not available 
dummy_readout__qubit1__reference__sensor1_I  :	Not available 
dummy_readout__qubit1__reference__sensor1_IQ :	Not available 
dummy_readout__qubit1__reference__sensor1_Q  :	Not available 
t_between_measurements                       :	50 (cycles)


#### 2.3.2 Observables

From the parameter name you can easily find the respective observable. Double underscores in the name indicate that the following name is an attribute of the previous class. Lets give it a try:

In [9]:
dummy_readout.qubit1.diff

In [10]:
dummy_readout.qubit1.reference.sensor1_IQ

Calling an observable returns you its `GettableParameter`:

In [11]:
dummy_readout.qubit1.diff()

<arbok_driver.gettable_parameter.GettableParameter: dummy_readout__qubit1__diff at 1556002745808>

#### 2.3.3 Signals

All present signales can be accquired by calling the signals attribute of the `ReadSequence`.

In [12]:
dummy_readout.signals

{'qubit1': <arbok_driver.signal.Signal object at 0x0000016A48FD9C90>}

#### 2.3.4 Readout points and abstract readouts

All `ReadoutPoints` of a sequence can be found but also the ones that are linked to a specific `Signal`:

In [13]:
dummy_readout.readout_points


{
    'qubit1__reference': <example_sequences.average_readout.Average object at 0x0000016A48FDAF80>,
    'qubit1__read': <example_sequences.average_readout.Average object at 0x0000016A48FD9510>
}

In [14]:
dummy_readout.qubit1.readout_points


{
    'qubit1__reference': <example_sequences.average_readout.Average object at 0x0000016A48FDAF80>,
    'qubit1__read': <example_sequences.average_readout.Average object at 0x0000016A48FD9510>
}

The same is true for `AbstractReadout`. However those are not necessarily signal specific thus they are not bound to a `Signal`.

In [15]:
dummy_readout.abstract_readouts

{'qubit1__diff': <example_sequences.difference.Difference object at 0x0000016A48FDAA70>}

`ReadoutPoint`s and `AbstractReadout`s can instroduce multiple `Observables`s (single results). The 'average' method being used in the given example introduces 3 observables:

In [16]:
dummy_readout.qubit1.reference.observables


{
    'dummy_readout__qubit1__reference__sensor1_I': <arbok_driver.observable.Observable object at 0x0000016A48FD9600>,
    'dummy_readout__qubit1__reference__sensor1_Q': <arbok_driver.observable.Observable object at 0x0000016A48FDA110>,
    'dummy_readout__qubit1__reference__sensor1_IQ': <arbok_driver.observable.Observable object at 0x0000016A48FDA170>
}

## 3. Compiling a `ReadSequence` to QUA code

After instanciating the `ReadSequence` it can be compiled to QUA code as shown below:

In [17]:
qua_program = qm_driver.get_qua_program()

qm_driver.print_qua_program_to_file(
    'qua_programs/tut2_readout.py', qua_program)
from qua_programs import tut2_readout
tut2_readout??

Type:        module
String form: <module 'qua_programs.tut2_readout' from 'C:\\Users\\z5440260\\GitRepos\\arbok_driver\\docs\\qua_programs\\tut2_readout.py'>
File:        c:\users\z5440260\gitrepos\arbok_driver\docs\qua_programs\tut2_readout.py
Source:     

# Single QUA script generated at 2024-04-12 13:25:52.707227
# QUA library version: 1.1.7

from qm.qua import *

with program() as prog:
    v1 = declare(fixed, )
    v2 = declare(fixed, )
    v3 = declare(fixed, )
    v4 = declare(fixed, )
    v5 = declare(fixed, )
    v6 = declare(fixed, )
    v7 = declare(fixed, )
    with infinite_loop_():
        pause()
        align()
        measure("measure", "readout_element", None, integration.full("x", v1, ""), integration.full("y", v2, ""))
        assign(v3, (v1+v2))
        r1 = declare_stream()
        save(v1, r1)
        r2 = declare_stream()
        save(v2, r2)
        r3 = declare_stream()
        save(v3, r3)
        wait(50, )
        measure("measure", "readout_element", None

As seen in the output, arbok takes care of all repetitive parts of the QUA program like declaring variables or streams. It only allocated memory for variables that are actually used within the measurement.

The user can therefore focus on writing the pulse sequences instead of managing FPGA resources.

## 4. Parameter sweeps and measurements 

Now we know how to set up sequences with measurements. Let us combine this with adding parameter sweeps as shown in tutorial 1. Just call the sequence and `set_sweeps` by given one dict per sweep axis with the parameter to sweep as key and the sweep array as value.

In [18]:
import numpy as np

In [19]:
dummy_sequence.set_sweeps(
    {
        dummy_readout.t_between_measurements: np.arange(10,100,10, dtype = int)
    },
)

In [20]:
qua_program = qm_driver.get_qua_program()
qm_driver.print_qua_program_to_file(
    'qua_programs/tut2_readout_with_sweep.py', qua_program)
from qua_programs import tut2_readout_with_sweep
tut2_readout_with_sweep??

C:\Users\z5440260\GitRepos\arbok_driver\arbok_driver\sweep.py:218: UserWarning: Your input array None for t_between_measurements will be parametrized, check output
  warnings.warn(


Type:        module
String form: <module 'qua_programs.tut2_readout_with_sweep' from 'C:\\Users\\z5440260\\GitRepos\\arbok_driver\\docs\\qua_programs\\tut2_readout_with_sweep.py'>
File:        c:\users\z5440260\gitrepos\arbok_driver\docs\qua_programs\tut2_readout_with_sweep.py
Source:     

# Single QUA script generated at 2024-04-12 13:25:52.902401
# QUA library version: 1.1.7

from qm.qua import *

with program() as prog:
    v1 = declare(int, )
    v2 = declare(fixed, )
    v3 = declare(fixed, )
    v4 = declare(fixed, )
    v5 = declare(fixed, )
    v6 = declare(fixed, )
    v7 = declare(fixed, )
    v8 = declare(fixed, )
    v9 = declare(int, )
    with infinite_loop_():
        pause()
        with for_(v1,10,(v1<100.0),(v1+10.0)):
            align()
            measure("measure", "readout_element", None, integration.full("x", v2, ""), integration.full("y", v3, ""))
            assign(v4, (v2+v3))
            r1 = declare_stream()
            save(v2, r1)
            r2 = declar

## 5. Scaling readout sequences to bigger systems

To use this type of readout on a bigger system, you only need to touch the configuration. All sequences are always meant to be designed to be sample agnostic. In the case below we add a second signal 'qubit2' with the same ReadPoints and the respective AbstractReadout to take the difference between those results. Scroll down to see the QUA program of this sequence.

In [21]:
readout_sequence_config2 = {
    'parameters': {
        't_between_measurements': {
            'value': 50,
            'unit': 'cycles',
        }
    },
    'signals':{
        'qubit1':{
            'elements': {
                'sensor1': 'readout_element',
            },
            'readout_points': {
                'reference': {
                    'method': 'average',
                    'desc':'reference point',
                    'observables': ['I', 'Q', 'IQ'],
                    'save_values': True
                },
                'read': {
                    'method': 'average',
                    'desc': 'redout point',
                    'observables': ['I', 'Q', 'IQ'],
                    'save_values': True
                }
            }
        },
        'qubit2':{
            'elements': {
                'sensor1': 'readout_element',
            },
            'readout_points': {
                'reference': {
                    'method': 'average',
                    'desc':'reference point',
                    'observables': ['I', 'Q', 'IQ'],
                    'save_values': True
                },
                'read': {
                    'method': 'average',
                    'desc': 'redout point',
                    'observables': ['I', 'Q', 'IQ'],
                    'save_values': True
                }
            }
        },
    },
    'readout_groups': {
        'difference': {
            'qubit1__diff': {
                'method': 'difference',
                'name': 'diff',
                'args': {
                    'signal': 'qubit1',
                    'minuend': 'qubit1.reference.sensor1_IQ',
                    'subtrahend': 'qubit1.read.sensor1_IQ',
                },
            },
            'qubit2__diff': {
                'method': 'difference',
                'name': 'diff',
                'args': {
                    'signal': 'qubit2',
                    'minuend': 'qubit2.reference.sensor1_IQ',
                    'subtrahend': 'qubit2.read.sensor1_IQ',
                },
            },
        }
    },
}

Creating the driver and sequences:

In [22]:
dummy_readout2 = DummyReadout('dummy_readout2', dummy_sample,
                              readout_sequence_config2)

In [23]:
qm_driver2 = ArbokDriver('qm_driver2', dummy_sample)
dummy_sequence2 = Sequence('dummy_squence2', dummy_sample)
qm_driver2.add_sequence(dummy_sequence2)
dummy_sequence2.add_subsequence(dummy_readout2)

Below see how the corresponding parameters were created without even touching the DummySequence. This example represents the philosophy of arbok:
- write device and setup agnostic control/read sequences
- quantify those sequences with configuration files

In [24]:
dummy_readout2.print_readable_snapshot()

dummy_readout2:
	parameter                                    value
--------------------------------------------------------------------------------
IDN                                           :	None 
dummy_readout2__qubit1__diff                  :	Not available 
dummy_readout2__qubit1__read__sensor1_I       :	Not available 
dummy_readout2__qubit1__read__sensor1_IQ      :	Not available 
dummy_readout2__qubit1__read__sensor1_Q       :	Not available 
dummy_readout2__qubit1__reference__sensor1_I  :	Not available 
dummy_readout2__qubit1__reference__sensor1_IQ :	Not available 
dummy_readout2__qubit1__reference__sensor1_Q  :	Not available 
dummy_readout2__qubit2__diff                  :	Not available 
dummy_readout2__qubit2__read__sensor1_I       :	Not available 
dummy_readout2__qubit2__read__sensor1_IQ      :	Not available 
dummy_readout2__qubit2__read__sensor1_Q       :	Not available 
dummy_readout2__qubit2__reference__sensor1_I  :	Not available 
dummy_readout2__qubit2__reference__sensor

QUA variables and streams are automatically put in place and the sequence is executed as expect for the system with double the size.

In [25]:
qua_program = qm_driver2.get_qua_program()

qm_driver.print_qua_program_to_file(
    'qua_programs/tut2_double_readout.py', qua_program)
from qua_programs import tut2_double_readout
tut2_double_readout??

Type:        module
String form: <module 'qua_programs.tut2_double_readout' from 'C:\\Users\\z5440260\\GitRepos\\arbok_driver\\docs\\qua_programs\\tut2_double_readout.py'>
File:        c:\users\z5440260\gitrepos\arbok_driver\docs\qua_programs\tut2_double_readout.py
Source:     

# Single QUA script generated at 2024-04-12 13:25:53.211391
# QUA library version: 1.1.7

from qm.qua import *

with program() as prog:
    v1 = declare(fixed, )
    v2 = declare(fixed, )
    v3 = declare(fixed, )
    v4 = declare(fixed, )
    v5 = declare(fixed, )
    v6 = declare(fixed, )
    v7 = declare(fixed, )
    v8 = declare(fixed, )
    v9 = declare(fixed, )
    v10 = declare(fixed, )
    v11 = declare(fixed, )
    v12 = declare(fixed, )
    v13 = declare(fixed, )
    v14 = declare(fixed, )
    with infinite_loop_():
        pause()
        align()
        measure("measure", "readout_element", None, integration.full("x", v1, ""), integration.full("y", v2, ""))
        assign(v3, (v1+v2))
        r1 = d